# Data Formats

In this research we use tabular representation of words in the text. This is not how text is represented in e.g. Hugging Face Transformers Models, where words are usually decomposed into the sequence of tokens. Tokens approach simply would provide us much more information than needed to solve the tasks we want, and we would need to deal with large datasets to avoid overfitting. Instead, we want to provide less precise information about each word, such as Pymorphy or Slovnet outputs. Hence, organizing this data in text is needed.

In this demo we show the set of classes that encapsulate the tedious preprocessing of text, such as separation into sentences/words and running well-known solutions on it. This is done under the hood, so after running a few routines we can get data in tabular format and dive directly into interesting tasks instead of peculiarities of NLP.

## Separator

Separator is the class that process the text and produces a dataframe.

In [1]:
from grammar_ru import Separator

text = 'Маги только думали, что могут контролировать информационную среду, но на самом деле все происходило по таким же биологическим законам, по которым рыбы в океане выбирают, куда им плыть. Это не люди выстраивали картину мира, а картина мира выстраивала себя через них. Бесполезно было искать виноватых.'
df = Separator.separate_string(text)
df.head()

,word_id,sentence_id,word_index,paragraph_id,word_tail,word,word_type,word_length
0,0,0,0,0,1,Маги,ru,4
1,1,0,1,0,1,только,ru,6
2,2,0,2,0,0,думали,ru,6
3,3,0,3,0,1,",",punct,1
4,4,0,4,0,1,что,ru,3


Columns' names are mostly self-explaining.
* `word_tail` is the amount of spaces that followed the word in the original text. `
* `word_id`, `sentence_id`, `paragraph_id` must be unique in the whole corpus.
* `word_index` is the position of the word inside the sentence.

Once separated, text can be viewed with `Separator.Viewer`

In [2]:
Separator.Viewer().to_text(df.loc[df.sentence_id==2])

'Бесполезно было искать виноватых.'

## Bundle

Bundle is a set of named dataframes, typically describing the same text.

Separator can run featurizers on the text, placing the output of each featurizer into one or several dataframes in the bundle:

In [3]:
from grammar_ru.features import PyMorphyFeaturizer, SlovnetFeaturizer

db = Separator.build_bundle(text, [PyMorphyFeaturizer(), SlovnetFeaturizer()])

In [4]:
list(db.data_frames)

['src', 'pymorphy', 'slovnet']

In [5]:
db.data_frames['src'].head()

,word_id,sentence_id,word_index,paragraph_id,word_tail,word,word_type,word_length
0,0,0,0,0,1,Маги,ru,4
1,1,0,1,0,1,только,ru,6
2,2,0,2,0,0,думали,ru,6
3,3,0,3,0,1,",",punct,1
4,4,0,4,0,1,что,ru,3


Data bundle allows **read-only** accessing of the dataframes:

In [6]:
db.src.head()

,word_id,sentence_id,word_index,paragraph_id,word_tail,word,word_type,word_length
0,0,0,0,0,1,Маги,ru,4
1,1,0,1,0,1,только,ru,6
2,2,0,2,0,0,думали,ru,6
3,3,0,3,0,1,",",punct,1
4,4,0,4,0,1,что,ru,3


In [7]:
db.pymorphy.head()

,normal_form,alternatives,score,delta_score,POS,animacy,gender,number,case,aspect,transitivity,person,tense,mood,voice,involvement
word_id,,,,,,,,,,,,,,,,
0,маг,1,1.000000,1.000000,NOUN,anim,masc,plur,nomn,NaN,NaN,NaN,NaN,NaN,None,None
1,только,3,0.500000,0.250000,ADVB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
2,думать,2,0.500000,0.000000,VERB,NaN,NaN,plur,NaN,impf,intr,NaN,past,indc,None,None
3,",",1,1.000000,1.000000,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
4,что,5,0.922033,0.891525,CONJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None


In [8]:
db.slovnet.head()

,POS,Animacy,Case,Gender,Number,Aspect,Mood,Tense,VerbForm,Voice,Person,Degree,Polarity,relation,syntax_parent_id
word_id,,,,,,,,,,,,,,,
0,NOUN,Anim,Nom,Fem,Plur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nsubj,2
1,PART,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,advmod,2
2,VERB,NaN,NaN,NaN,Plur,Imp,Ind,Past,Fin,Act,NaN,NaN,NaN,root,-1
3,PUNCT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,punct,5
4,SCONJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mark,5


Once text was featurized, basic statistical research may be performed with the `pandas` library. 

`Separator.Viewer` can additionaly be used to highlight the words in the text:

In [9]:
df = db.src.merge(db.pymorphy[['POS']], left_on='word_id', right_index=True)
Separator.Viewer().highlight('POS','auto').tooltip('POS').to_html_display(df.loc[df.sentence_id==1])

## Corpus

Corpus is a set of bundles, each representing one "atomic" text in the collection, e.g. a chapter in the book. "Atomic" means that the statistical operations we perform on this text can be performed within this one bundle, and we don't need to join across the bundles. Separation of texts into bundles is important to control the memory consumption: sometimes we need e.g. to merge dataframes with themselves, and such operations are too memory-intensive to be performed on the whole books. 

Corpuses are build from "pseudo-md" files that contain:

* Headers, starting with `#`
* Tags, starting with `$`
* Raw text

When building corpus from e.g. HTML files, those must first be converted into pseudomd format, and then fed to the `grammar_ru` pipelines. The advantage of this approach is that such pseudomd files can be reviewed manually.

`grammar_ru` also contains the auxiliary code to convert `fb2` format into pseudomd.

This is an example of md-file:

In [10]:
from grammar_ru.common import FileIO

print(FileIO.read_text('files/mds/averchenko.md')[:200]+'...')

# Баклуши

$ {"rating":10}

Севастополь.
День 15 июля.
Весь город шумит, гремит, дымит, грохочет, деловые люди снуют, кричат, торгуют, магазины широко открыли свои гостеприимные двери — будничная дело...


This is how conversion works:

* First, we convert the `md` files inside a specified folder into a base corpus
* Second, we run the featurizers on the base corpus, producing the featurized corpus.

Both corpuses are `zip`-files that are easy to spread around

In [11]:
from grammar_ru.corpus import CorpusBuilder
from pathlib import Path

corpus = Path('files/corpus.zip')
base_corpus = Path('files/corpus.base.zip')


CorpusBuilder.convert_interformat_folder_to_corpus(
    base_corpus,
    Path('files/mds'),
    ['author']
)

CorpusBuilder.featurize_corpus(
    base_corpus,
    corpus,
    [
        PyMorphyFeaturizer(),
        SlovnetFeaturizer()
    ]
)


  0%|          | 0/2 [00:00<?, ?it/s]

2024-03-30 13:56:21.211262 INFO: Processing 9cb16747-a24b-4496-a843-7b03bf7148da at #0, total 3 
2024-03-30 13:56:21.212262 INFO: <class 'grammar_ru.features.pymorphy_featurizer.PyMorphyFeaturizer'>
2024-03-30 13:56:21.230390 INFO: <class 'grammar_ru.features.slovnet_featurizer.SlovnetFeaturizer'>
2024-03-30 13:56:21.283390 INFO: Processing 64e69c7a-c363-4061-bb9b-b2be67c1313e at #1, total 3 
2024-03-30 13:56:21.283390 INFO: <class 'grammar_ru.features.pymorphy_featurizer.PyMorphyFeaturizer'>
2024-03-30 13:56:21.327390 INFO: <class 'grammar_ru.features.slovnet_featurizer.SlovnetFeaturizer'>
2024-03-30 13:56:21.410390 INFO: Processing 599a3988-e1fd-4d92-b2fe-06647a3d8611 at #2, total 3 
2024-03-30 13:56:21.410390 INFO: <class 'grammar_ru.features.pymorphy_featurizer.PyMorphyFeaturizer'>
2024-03-30 13:56:21.516390 INFO: <class 'grammar_ru.features.slovnet_featurizer.SlovnetFeaturizer'>


Corpus contains table-of-contents (toc) file, describing all bundles in the corpus: basic statistics, headers and tags:

In [13]:
from grammar_ru import CorpusReader

reader = CorpusReader(corpus)
reader.get_toc()

,filename,timestamp,part_index,token_count,character_count,ordinal,header_0,headers,tag_rating,author,max_id
file_id,,,,,,,,,,,
9cb16747-a24b-4496-a843-7b03bf7148da,averchenko.md,2024-03-30 13:56:20.770253,0,204,865,0,Баклуши,Баклуши,10.0,averchenko,205
64e69c7a-c363-4061-bb9b-b2be67c1313e,averchenko.md,2024-03-30 13:56:20.781253,1,453,2035,1,Белые короли,Белые короли,20.0,averchenko,10659
599a3988-e1fd-4d92-b2fe-06647a3d8611,chekhov.md,2024-03-30 13:56:20.821253,0,1422,6010,2,Ванька,Ванька,NaN,chekhov,22082


`reader.read_frames()` provides an iterator with the `src` frames of each bundle:

In [14]:
reader.read_frames().first().head()

,word_id,sentence_id,word_index,paragraph_id,word_tail,word,word_type,word_length,file_id,corpus_id
0,0,0,0,0,0,Севастополь,ru,11,9cb16747-a24b-4496-a843-7b03bf7148da,corpus.zip
1,1,0,1,0,0,.,punct,1,9cb16747-a24b-4496-a843-7b03bf7148da,corpus.zip
2,2,1,0,1,1,День,ru,4,9cb16747-a24b-4496-a843-7b03bf7148da,corpus.zip
3,3,1,1,1,1,15,unk,2,9cb16747-a24b-4496-a843-7b03bf7148da,corpus.zip
4,4,1,2,1,0,июля,ru,4,9cb16747-a24b-4496-a843-7b03bf7148da,corpus.zip


`reader.read_bundles()` provides an iterator that reads the whole bundles:

In [15]:
reader.read_bundles().first()

{'src': {'shape': (204, 10), 'index_name': None}, 'pymorphy': {'shape': (204, 16), 'index_name': 'word_id'}, 'slovnet': {'shape': (204, 15), 'index_name': 'word_id'}}